# Modelado

### Importacion de librerias

In [3]:
import pandas as pd
import numpy as np

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [5]:
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns

### Carga de datos

In [7]:
df = pd.read_csv("../data/processed/reclamos_descripcionesv2.csv", low_memory=False)

In [8]:
df.head()

,ID_PERIODO,DE_TIPO_ADMINISTRADO,CO_ADMIN_DECLA,CO_UGIPRESS,DE_TIPO_INSTITUCION,CO_ADMIN_SUCE,DE_MEDIO_PRESENTACION,CO_UNICO_RECLAMO,DE_MEDIO_RECEPCION,FE_PRESEN_RECLA,...,DE_TIPO_ADMIN_DERIVA,CO_ADMIN_DERIVA,DE_RESULTADO,DE_MOTIVO_CONCL_ANTIC,FE_RESULT_RECL,DE_COMUN_RESULT,FE_NOTIFI_RESULT,AÑO,MES,DESCRIPCION
0,202101,IAFAS,20007,20007,IAFAS,20007,Físico,20007-352,Libro de Reclamaciones Físico,2020-11-27,...,-,-,Infundado,-,2020-12-21,Correo electrónico,NaN,2020,11,-
1,202101,IAFAS,20007,20007,IAFAS,20007,Físico,20007-360,Libro de Reclamaciones Físico,2021-01-04,...,-,-,Infundado,-,2021-02-18,Correo electrónico,NaN,2021,1,Me cobraron por una vacuna que estaba incluida...
2,202101,IAFAS,20007,20007,IAFAS,20007,Físico,20007-361,Libro de Reclamaciones Físico,2021-01-06,...,-,-,Infundado,-,2021-01-20,Correo electrónico,20210122.0,2021,1,Las ambulancias de EsSalud no llegaron a tiemp...
3,202101,IAFAS,20007,20007,IAFAS,20007,Físico,20007-364,Libro de Reclamaciones Físico,2021-01-06,...,-,-,Infundado,-,2021-01-25,Correo electrónico,20210204.0,2021,1,La atención que brindan las IAFAS no ha sido d...
4,202101,IAFAS,20007,20007,IAFAS,20007,Físico,20007-365,Libro de Reclamaciones Físico,2021-01-08,...,-,-,Pendiente,-,NaN,-,NaN,2021,1,He tenido problemas para acceder a mis servici...


### Limpiar y validar el dataset

In [10]:
print(f"Dimensiones del dataset: {df.shape}")
print("\nColumnas disponibles:")
print(df.columns.tolist())
print("\nInformación de tipos de datos:")
print(df.info())
print("\nEstadísticas descriptivas:")
print(df.describe())
print("\nValores faltantes por columna:")
print(df.isnull().sum())
print("\nDistribución de la variable objetivo:")
print(df['DE_CLASIF_1'].value_counts(normalize=True))

# Verificar duplicados
duplicados = df.duplicated().sum()
print(f"\nFilas duplicadas: {duplicados}")

Dimensiones del dataset: (162639, 28)

Columnas disponibles:
['ID_PERIODO', 'DE_TIPO_ADMINISTRADO', 'CO_ADMIN_DECLA', 'CO_UGIPRESS', 'DE_TIPO_INSTITUCION', 'CO_ADMIN_SUCE', 'DE_MEDIO_PRESENTACION', 'CO_UNICO_RECLAMO', 'DE_MEDIO_RECEPCION', 'FE_PRESEN_RECLA', 'DE_SERVICIO', 'DE_COMPETENCIA', 'DE_CLASIF_1', 'DE_CLASIF_2', 'DE_CLASIF_3', 'DE_ESTADO_RECLAMO', 'CO_RECLAMO_PRIMIG', 'DE_ETAPA_RECLAMO', 'DE_TIPO_ADMIN_DERIVA', 'CO_ADMIN_DERIVA', 'DE_RESULTADO', 'DE_MOTIVO_CONCL_ANTIC', 'FE_RESULT_RECL', 'DE_COMUN_RESULT', 'FE_NOTIFI_RESULT', 'AÑO', 'MES', 'DESCRIPCION']

Información de tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162639 entries, 0 to 162638
Data columns (total 28 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   ID_PERIODO             162639 non-null  int64 
 1   DE_TIPO_ADMINISTRADO   162639 non-null  object
 2   CO_ADMIN_DECLA         162639 non-null  int64 
 3   CO_UGIPRESS            

In [11]:
df_clean = df.copy()

In [12]:
print(f"Valores nulos en DESCRIPCION: {df_clean['DESCRIPCION'].isnull().sum()}")

Valores nulos en DESCRIPCION: 0


In [13]:
print(f"Valores con '-' en DESCRIPCION: {(df_clean['DESCRIPCION'] == '-').sum()}")

Valores con '-' en DESCRIPCION: 35200


In [14]:
# Rellenar valores faltantes en DESCRIPCION con un valor por defecto
df_clean['DESCRIPCION'] = df_clean['DESCRIPCION'].fillna("Sin descripción")

In [15]:
df_clean['DESCRIPCION'] = df_clean['DESCRIPCION'].replace('-', np.nan).fillna("Sin descripción")

In [ ]:
n_duplicados = df_clean.duplicated().sum()
if n_duplicados > 0:
    print(f"Eliminando {n_duplicados} filas duplicadas")
    df_clean = df_clean.drop_duplicates()

### Eliminar columnas que no debemos usar (data leakage)

In [18]:
columnas_data_leakage = [
    'DE_CLASIF_2', 'DE_CLASIF_3',
    'DE_ESTADO_RECLAMO',
    'CO_RECLAMO_PRIMIG',
    'DE_ETAPA_RECLAMO',
    'DE_TIPO_ADMIN_DERIVA',
    'CO_ADMIN_DERIVA',
    'DE_RESULTADO',
    'DE_MOTIVO_CONCL_ANTIC',
    'FE_RESULT_RECL',
    'DE_COMUN_RESULT',
    'FE_NOTIFI_RESULT',
    'DE_TIPO_ADMINISTRADO'
]

In [19]:
# Verificar cuáles de estas columnas existen en el dataset
columnas_existentes = [col for col in columnas_data_leakage if col in df.columns]

In [20]:
if columnas_existentes:
    print(f"Eliminando columnas por data leakage: {columnas_existentes}")
    df_clean = df_clean.drop(columns=columnas_existentes)

Eliminando columnas por data leakage: ['DE_CLASIF_2', 'DE_CLASIF_3', 'DE_ESTADO_RECLAMO', 'CO_RECLAMO_PRIMIG', 'DE_ETAPA_RECLAMO', 'DE_TIPO_ADMIN_DERIVA', 'CO_ADMIN_DERIVA', 'DE_RESULTADO', 'DE_MOTIVO_CONCL_ANTIC', 'FE_RESULT_RECL', 'DE_COMUN_RESULT', 'FE_NOTIFI_RESULT', 'DE_TIPO_ADMINISTRADO']


### Procesar variables temporales

In [22]:
df_clean['FE_PRESEN_RECLA']

0         2020-11-27
1         2021-01-04
2         2021-01-06
3         2021-01-06
4         2021-01-08
             ...    
162634    2024-06-29
162635    2024-06-14
162636    2024-06-17
162637    2024-06-14
162638    2024-06-17
Name: FE_PRESEN_RECLA, Length: 162639, dtype: object

In [23]:
df_clean['FE_PRESEN_RECLA'].dtype

dtype('O')

In [24]:
if 'FE_PRESEN_RECLA' in df_clean.columns:
    df_clean['FE_PRESEN_RECLA'] = pd.to_datetime(df_clean['FE_PRESEN_RECLA'], format='%Y-%m-%d', errors='coerce')
    df_clean['MES'] = df_clean['FE_PRESEN_RECLA'].dt.month
    df_clean['DIA_SEMANA'] = df_clean['FE_PRESEN_RECLA'].dt.dayofweek
    df_clean['AÑO'] = df_clean['FE_PRESEN_RECLA'].dt.year
    df_clean['DIA_MES'] = df_clean['FE_PRESEN_RECLA'].dt.day
    registros_invalidos = df_clean['FE_PRESEN_RECLA'].isnull().sum()
    if registros_invalidos > 0:
        print(f"Se encontraron {registros_invalidos} registros con fechas inválidas")

In [25]:
df_clean['FE_PRESEN_RECLA']

0        2020-11-27
1        2021-01-04
2        2021-01-06
3        2021-01-06
4        2021-01-08
            ...    
162634   2024-06-29
162635   2024-06-14
162636   2024-06-17
162637   2024-06-14
162638   2024-06-17
Name: FE_PRESEN_RECLA, Length: 162639, dtype: datetime64[ns]

In [26]:
if 'ID_PERIODO' in df_clean.columns and df_clean['ID_PERIODO'].dtype == 'object':
    try:
        df_clean['PERIODO_AÑO'] = df_clean['ID_PERIODO'].str[:4].astype(int)
        df_clean['PERIODO_MES'] = df_clean['ID_PERIODO'].str[4:6].astype(int)
    except:
        print("Error al procesar ID_PERIODO")

In [27]:
print("\nDistribución de clases en DE_CLASIF_1:")
print(df_clean['DE_CLASIF_1'].value_counts(normalize=True))


Distribución de clases en DE_CLASIF_1:
DE_CLASIF_1
-                                                                                   0.389015
Otros relativos a las IAFAS                                                         0.271091
Negar la acreditación de usuario asegurado.                                         0.065206
Cobrar indebidamente                                                                0.065138
Negar  o demora en otorgar la cobertura en salud                                    0.055971
No permitir al usuario la libre elección de IPRESS de acuerdo a lo contratado       0.051703
No brindar información sobre sus derechos en salud                                  0.036805
Demorar la gestión de la carta de garantía y/o reembolsos.                          0.032428
Negar o demorar en la atención en la IAFAS                                          0.012469
Negar atención para el trámite de registro o acreditación                           0.005810
No brindar atenció

In [28]:
X = df_clean.drop(columns=['DE_CLASIF_1'])
y = df_clean['DE_CLASIF_1']

In [ ]:
X_train, X_test, y_train, y_test =  (X, y, test_size=0.2, stratify=y, random_state=42)

In [76]:
print(f"Tamaño entrenamiento: {X_train.shape}")
print(f"Tamaño prueba: {X_test.shape}")

Tamaño entrenamiento: (130111, 16)
Tamaño prueba: (32528, 16)


In [31]:
cat_bajo_cardinal = ['DE_TIPO_ADMINISTRADO', 'DE_TIPO_INSTITUCION', 
                     'DE_MEDIO_PRESENTACION', 'DE_MEDIO_RECEPCION',
                     'DE_SERVICIO', 'DE_COMPETENCIA']
cat_alto_cardinal = ['CO_ADMIN_DECLA', 'CO_ADMIN_SUCE', 'CO_UGIPRESS']
numericas = ['MES', 'DIA_SEMANA', 'AÑO', 'DIA_MES', 'PERIODO_AÑO', 'PERIODO_MES']

In [32]:
cat_bajo_cardinal = [col for col in cat_bajo_cardinal if col in X_train.columns]
cat_alto_cardinal = [col for col in cat_alto_cardinal if col in X_train.columns]
numericas = [col for col in numericas if col in X_train.columns]

In [33]:
import nltk

In [34]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\harol\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [35]:
from nltk.corpus import stopwords

In [36]:
stop_words_spanish = stopwords.words('spanish')

In [37]:
# Asegurarte de que las columnas categóricas de baja y alta cardinalidad sean tipo texto
for col in cat_bajo_cardinal:
    if df[col].dtype != 'object':
        print(f"⚠️  {col} no es tipo object. Convertir a string.")
        df[col] = df[col].astype(str)

In [38]:
for col in cat_alto_cardinal:
    if df[col].dtype != 'object':
        print(f"⚠️  {col} no es tipo object. Convertir a string.")
        df[col] = df[col].astype(str)

⚠️  CO_ADMIN_DECLA no es tipo object. Convertir a string.
⚠️  CO_ADMIN_SUCE no es tipo object. Convertir a string.
⚠️  CO_UGIPRESS no es tipo object. Convertir a string.


In [39]:
# Forzar a tipo string las columnas categóricas que no son 'object'
for col in cat_bajo_cardinal + cat_alto_cardinal:
    if df[col].dtype != 'object':
        df[col] = df[col].astype(str)


In [40]:
print(X_train[cat_bajo_cardinal + cat_alto_cardinal].dtypes)


DE_TIPO_INSTITUCION      object
DE_MEDIO_PRESENTACION    object
DE_MEDIO_RECEPCION       object
DE_SERVICIO              object
DE_COMPETENCIA           object
CO_ADMIN_DECLA            int64
CO_ADMIN_SUCE             int64
CO_UGIPRESS               int64
dtype: object


In [41]:
for col in cat_bajo_cardinal + cat_alto_cardinal:
    X_train[col] = X_train[col].astype(str)
    X_test[col] = X_test[col].astype(str)


In [42]:
transformadores = []

In [43]:
if 'DESCRIPCION' in X_train.columns:
    transformadores.append(('text', TfidfVectorizer(stop_words=stop_words_spanish, max_features=3000, ngram_range=(1,2), min_df=5), 'DESCRIPCION'))

In [44]:
if cat_bajo_cardinal:
    transformadores.append(('cat_low', Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ]), cat_bajo_cardinal))

In [45]:
if cat_alto_cardinal:
    transformadores.append(('cat_high', Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
    ]), cat_alto_cardinal))

In [46]:
if numericas:
    transformadores.append(('num', Pipeline([
        ('imputer', SimpleImputer(strategy='median'))
    ]), numericas))

In [47]:
preprocessor = ColumnTransformer(transformers=transformadores, remainder='drop')

In [48]:
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier


In [52]:
# Xgboost
from sklearn.preprocessing import LabelEncoder

# Codificar las etiquetas (y)
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [54]:
# Xgboost
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(
        objective='multi:softprob',  # útil para clasificación multiclase
        eval_metric='mlogloss',      # buena métrica para esto
        use_label_encoder=False,
        random_state=42,
        n_jobs=-1
    ))
])

In [50]:
# pipeline = Pipeline([
#     ('preprocessor', preprocessor),
#     ('classifier', RandomForestClassifier(
#         n_estimators=100,
#         class_weight='balanced',
#         random_state=42
#     ))
# ])

In [56]:
print("Entrenando modelo...")
pipeline.fit(X_train, y_train_encoded)
print("Modelo entrenado.")

Entrenando modelo...


C:\Users\harol\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:04:24] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Modelo entrenado.


In [64]:
#y_pred = pipeline.predict(X_test)

In [68]:
#xgboost
y_pred_encoded = pipeline.predict(X_test)

In [70]:
# Decodificar predicciones
y_pred = label_encoder.inverse_transform(y_pred_encoded)

In [72]:
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))


Reporte de clasificación:
                                                                                  precision    recall  f1-score   support

                                                                               -       1.00      1.00      1.00     12654
                                                            Cobrar indebidamente       1.00      1.00      1.00      2119
                      Demorar la gestión de la carta de garantía y/o reembolsos.       0.86      0.87      0.86      1055
                                Negar  o demora en otorgar la cobertura en salud       0.79      0.80      0.79      1821
                       Negar atención para el trámite de registro o acreditación       1.00      1.00      1.00       189
                     Negar el otorgamiento de prestaciones económicas o sociales       1.00      1.00      1.00        47
                                     Negar la acreditación de usuario asegurado.       0.88      0.87      0.88      2

In [ ]:
plt.figure(figsize=(12, 10))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=sorted(set(y_test)), yticklabels=sorted(set(y_test)))
plt.title('Matriz de Confusión')
plt.ylabel('Real')
plt.xlabel('Predicho')
plt.tight_layout()
plt.show()

In [80]:
%pip install tensorflow

   ---------------------------------------- 0.0/376.0 MB ? eta -:--:--
   ---------------------------------------- 2.4/376.0 MB 11.2 MB/s eta 0:00:34
   ---------------------------------------- 4.2/376.0 MB 10.1 MB/s eta 0:00:37
    --------------------------------------- 6.0/376.0 MB 9.5 MB/s eta 0:00:40
    --------------------------------------- 7.6/376.0 MB 9.2 MB/s eta 0:00:40
   - -------------------------------------- 9.4/376.0 MB 9.0 MB/s eta 0:00:41
   - -------------------------------------- 11.3/376.0 MB 9.2 MB/s eta 0:00:40
   - -------------------------------------- 13.6/376.0 MB 9.3 MB/s eta 0:00:39
   - -------------------------------------- 17.8/376.0 MB 10.6 MB/s eta 0:00:34
   -- ------------------------------------- 22.5/376.0 MB 12.0 MB/s eta 0:00:30
   -- ------------------------------------- 27.5/376.0 MB 13.1 MB/s eta 0:00:27
   --- ------------------------------------ 32.5/376.0 MB 14.1 MB/s eta 0:00:25
   ---- ----------------------------------- 37.7/376.0 MB 1

In [82]:
#Redes
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.utils import to_categorical

# Label encoding
label_encoder = LabelEncoder()
y_train_int = label_encoder.fit_transform(y_train)
y_test_int = label_encoder.transform(y_test)

# One-hot encoding
y_train_cat = to_categorical(y_train_int)
y_test_cat = to_categorical(y_test_int)


In [84]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))


C:\Users\harol\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [88]:
print(X_train.dtypes)


ID_PERIODO                        int64
CO_ADMIN_DECLA                    int64
CO_UGIPRESS                       int64
DE_TIPO_INSTITUCION              object
CO_ADMIN_SUCE                     int64
DE_MEDIO_PRESENTACION            object
CO_UNICO_RECLAMO                 object
DE_MEDIO_RECEPCION               object
FE_PRESEN_RECLA          datetime64[ns]
DE_SERVICIO                      object
DE_COMPETENCIA                   object
AÑO                               int32
MES                               int32
DESCRIPCION                      object
DIA_SEMANA                        int32
DIA_MES                           int32
dtype: object


In [90]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train_cat, epochs=20, batch_size=32, validation_split=0.2)


ValueError: Invalid dtype: object

In [ ]:
# Predicciones
y_pred_prob = model.predict(X_test)
y_pred_int = y_pred_prob.argmax(axis=1)
y_pred = label_encoder.inverse_transform(y_pred_int)

# Reporte
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))
